# Line Plots

## 2D line plots

This example plots a sine wave, cosine wave, and ricker wavelet and demonstrates how Graphic Features can be modified by slicing!

In [1]:
import fastplotlib as fpl
from fastplotlib.graphics.selectors import Synchronizer

import numpy as np
from sidecar import Sidecar
from ipywidgets import VBox
from itertools import product

### First generate some data.

In [2]:
# linspace, create 100 evenly spaced x values from -10 to 10
xs = np.linspace(-10, 10, 100)
# sine wave
ys = np.sin(xs)
sine = np.dstack([xs, ys])[0]

# cosine wave
ys = np.cos(xs) + 5
cosine = np.dstack([xs, ys])[0]

# sinc function
a = 0.5
ys = np.sinc(xs) * 3 + 8
sinc = np.dstack([xs, ys])[0]

### We will plot all of it on the same plot. Each line plot will be an individual Graphic, you can have any combination of graphics on a plot.

In [3]:
# Create a plot instance
plot_l = fpl.Plot()

# plot sine wave, use a single color
sine_graphic = plot_l.add_line(data=sine, thickness=5, colors="magenta")

# you can also use colormaps for lines!
cosine_graphic = plot_l.add_line(data=cosine, thickness=12, cmap="autumn")

# or a list of colors for each datapoint
colors = ["r"] * 25 + ["purple"] * 25 + ["y"] * 25 + ["b"] * 25
sinc_graphic = plot_l.add_line(data=sinc, thickness=5, colors = colors)

sc = Sidecar(title="lines")

with sc: 
    display(plot_l.show())

RFBOutputContext()

/home/clewis7/repos/fastplotlib/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")


### "stretching" the camera, useful for large timeseries data

Set `maintain_aspect = False` on a camera, and then use the right mouse button and move the mouse to stretch and squeeze the view!

You can also click the `1:1` button to toggle this.

In [4]:
plot_l.camera.maintain_aspect = False

### reset the plot area

In [5]:
plot_l.auto_scale(maintain_aspect=True)

### Graphic features support slicing! :D

In [6]:
# indexing of colors
cosine_graphic.colors[:15] = "magenta"

In [7]:
cosine_graphic.colors[90:] = "red"

In [8]:
cosine_graphic.colors[60] = "w"

In [9]:
# indexing to assign colormaps to entire lines or segments
sinc_graphic.cmap[10:50] = "gray"
sine_graphic.cmap = "seismic"

In [10]:
# more complex indexing, set the blue value directly from an array
cosine_graphic.colors[65:90, 0] = np.linspace(0, 1, 90-65)

### You can capture changes to a graphic feature as events

In [11]:
def callback_func(event_data):
    print(event_data)

# Will print event data when the color changes
cosine_graphic.colors.add_event_handler(callback_func)

In [12]:
# more complex indexing of colors
# from point 15 - 30, set every 3rd point as "cyan"
cosine_graphic.colors[15:50:3] = "cyan"

FeatureEvent @ 0x7ff8f010e4d0
type: colors
pick_info: {'index': range(15, 50, 3), 'collection-index': None, 'world_object': <weakproxy at 0x7ff9063da340 to Line at 0x7ff906436e50>, 'new_data': array([[0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.]], dtype=float32)}



### Graphic data is itself also indexable

In [13]:
cosine_graphic.data[10:50:5, :2] = sine[10:50:5]

In [14]:
cosine_graphic.data[90:, 1] = 7

In [15]:
cosine_graphic.data[0] = np.array([[-10, 0, 0]])

/home/clewis7/repos/fastplotlib/fastplotlib/graphics/_features/_base.py:31: UserWarning: converting int64 array to int32
  warn(f"converting {array.dtype} array to int32")


In [20]:
plot_l.close()
sc.close()

### Can also have fancy indexing of colormaps

In [21]:
plot = fpl.Plot()

plot.add_line(sine, thickness=10, name="sine")

sc = Sidecar(title="line plot")

with sc:
    display(plot.show())

RFBOutputContext()

In [22]:
plot["sine"].cmap = "jet"

In [23]:
plot["sine"].cmap.values = sine[:, 1]

In [24]:
plot["sine"].cmap.values = cosine[:, 1]

In [25]:
plot["sine"].cmap = "viridis"

In [26]:
cmap_values = [0] * 25 + [1] * 5 + [2] * 50 + [3] * 20
plot["sine"].cmap.values = cmap_values

In [27]:
plot.close()
sc.close()

## Line Collections

Generate some data

In [28]:
def make_circle(center, radius: float, n_points: int = 75) -> np.ndarray:
    theta = np.linspace(0, 2 * np.pi, n_points)
    xs = radius * np.sin(theta)
    ys = radius * np.cos(theta)

    return np.column_stack([xs, ys]) + center


spatial_dims = (50, 50)

# this makes 16 circles, so we can create 16 cmap values, so it will use these values to set the
# color of the line based by using the cmap as a LUT with the corresponding cmap_value
circles = list()
for center in product(range(0, spatial_dims[0], 15), range(0, spatial_dims[1], 15)):
    circles.append(make_circle(center, 5, n_points=75))

pos_xy = np.vstack(circles)

### set cmap values as LUT

In [29]:
plot = fpl.Plot()

# highest values, lowest values, mid-high values, mid values
cmap_values = cmap_values = [
    0, 1, 1, 2,
    0, 0, 1, 1,
    2, 2, 3, 3,
    1, 1, 1, 5
]

plot.add_line_collection(circles, cmap="tab10", cmap_values=cmap_values, thickness=5)

sc = Sidecar(title="lines collection")

with sc:
    display(plot.show())

RFBOutputContext()

In [30]:
plot.close()
sc.close()

### can also set colors

In [31]:
plot = fpl.Plot()

colors = ["green"] * 4 + ["red"] * 4 + ["yellow"] * 4 + ["w"] * 4

plot.add_line_collection(circles, colors=colors, thickness=5)

sc = Sidecar(title="lines collection")

with sc:
    display(plot.show())

RFBOutputContext()

In [32]:
plot.close()
sc.close()

## Line Stack

Generate some data

In [33]:
xs = np.linspace(0, 100, 1000)
# sine wave
ys = np.sin(xs) * 20

# make 25 lines
data = np.vstack([ys] * 25)

In [34]:
plot = fpl.Plot()

# line stack takes all the same arguments as line collection and behaves similarly
plot.add_line_stack(data, cmap="jet")

sc = Sidecar(title="line stack")

with sc:
    display(plot.show(maintain_aspect=False))

RFBOutputContext()

In [35]:
plot.close()
sc.close()

## Linear Region Selector

Creates a linear region bounded graphic which can be moved along either the x-axis or y-axis.
Allows sub-selecting data from a `Graphic` or from multiple `Graphic` objects.

In [36]:
def interpolate(subdata: np.ndarray, axis: int):
    """1D interpolation to display within the preallocated data array"""
    x = np.arange(0, zoomed_prealloc)
    xp = np.linspace(0, zoomed_prealloc, subdata.shape[0])
    
    # interpolate to preallocated size
    return np.interp(x, xp, fp=subdata[:, axis])  # use the y-values

In [37]:
# data to plot
xs = np.linspace(0, 100, 1_000)
sine = np.sin(xs) * 20
cosine = np.cos(xs) * 20

plot = fpl.GridPlot((5, 1))

# preallocated size for zoomed data
zoomed_prealloc = 1_000
zoomed_init = np.column_stack([np.arange(zoomed_prealloc), np.random.rand(zoomed_prealloc)])

# sines and cosines
sines = [sine] * 2
cosines = [cosine] * 2

# make line stack
line_stack = plot[0, 0].add_line_stack(sines + cosines, separation=50)

# make selector
selector = line_stack.add_linear_region_selector()

# populate subplots with preallocated graphics
for i, subplot in enumerate(plot):
    if i == 0:
        # skip the first one
        continue
    # make line graphics for displaying zoomed data
    subplot.add_line(zoomed_init, name="zoomed")


def update_zoomed_subplots(ev):
    """update the zoomed subplots"""
    zoomed_data = selector.get_selected_data()
    
    for i in range(len(zoomed_data)):
        data = interpolate(zoomed_data[i], axis=1)
        plot[i + 1, 0]["zoomed"].data = data
        plot[i + 1, 0].auto_scale()


selector.selection.add_event_handler(update_zoomed_subplots)
plot.show()
sc = Sidecar(title="linear region selector")

with sc: 
    display(plot.show())

RFBOutputContext()

In [38]:
plot.close()
sc.close()

## Heatmap

In [39]:
temporal = np.load('/home/clewis7/repos/fastplotlib-scipy2023/data/temporal.npy')

In [40]:
# create plot
plot = fpl.GridPlot(shape=(1,2), names=[["heatmap", "traces"]])

# add temporal traces as heatmap
heatmap = plot[0,0].add_heatmap(temporal)

# add temporal traces as line stack
temporal_stack = plot[0,1].add_line_stack(temporal, colors="magenta")

sc = Sidecar(title="heatmap interactive")

with sc:
    display(plot.show())

RFBOutputContext()

In [41]:
plot.close()
sc.close()